<a href="https://colab.research.google.com/github/apurvasinhaa/html-portfolio/blob/main/13_FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset

id: unique id for news article

title: the title of a news article

author: author of the new article

text: the text of the article, could be incomplete

label:a label that marks whether the news article is real or fake
1--> Fake news 2--> Real news

Importing the Dependencies

re- regular expression

nltk- natural language tool kit



In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [ ]:
#load the data to pandas data frame
news_data = pd.read_csv('/content/train.csv', engine='python', on_bad_lines='warn')

<ipython-input-39-4dfcaa524200>:2: ParserWarning: Skipping line 6436: unexpected end of data

  news_data = pd.read_csv('/content/train.csv', engine='python', on_bad_lines='warn')


In [ ]:
#first 5 rows of data frame
news_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


0-->real news
1--> fake news

In [ ]:
news_data.shape

(6434, 5)

In [ ]:
#checking for missing values
news_data.isnull().sum()

,0
id,0
title,178
author,619
text,13
label,0


In [ ]:
#replacing missing values with null string
news_data = news_data.fillna('')

In [ ]:
#merge author name and new title
news_data['content'] = news_data['author']+' '+news_data['title']

In [ ]:
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
#separating feature and target (feature-->content & target-->label)
x=news_data.drop(columns='label',axis=1)
y=news_data['label']

In [ ]:
print(x)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
6429  6429  Moveable Feast Cafe 2016/10/28 … Open Thread |...   
6430  6430  Hajj Prep: Search Soul, Buy Sturdy Shoes, Pay ...   
6431  6431  10 radikale Vorhaben, die Donald Trump sofort ...   
6432  6432                               Little Blue Balls...   
6433  6433  WATCH: Louis CK’s EPIC Answer on Trump vs. Hil...   

                                   author  \
0                           Darrell Lucus   
1                         Daniel J. Flynn   
2                      Consortiumnews.com   
3                         Jessica Purki

In [ ]:
print(y)

0       1
1       0
2       1
3       1
4       1
       ..
6429    1
6430    0
6431    1
6432    1
6433    1
Name: label, Length: 6434, dtype: int64


Stemming

Stemming is the process of reducing a word to its rootword

exapmle : actor ,actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  content=str(content)
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_data['content'] = news_data['content'].apply(stemming)

In [ ]:
print(news_data['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
6429    herb swanson moveabl feast cafe open thread vi...
6430    diaa hadid hajj prep search soul buy sturdi sh...
6431    norepli blogger com der postillon radikal vorh...
6432                              recrisp littl blue ball
6433    grant stern watch loui ck epic answer trump vs...
Name: content, Length: 6434, dtype: object


In [ ]:
x= news_data['content'].values
y= news_data['label'].values


In [ ]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'norepli blogger com der postillon radikal vorhaben die donald trump sofort nach amtsantritt durchsetzen wird'
 'recrisp littl blue ball'
 'grant stern watch loui ck epic answer trump vs hillari']


In [ ]:
print(y)

[1 0 1 ... 1 1 1]


In [ ]:
y.shape

(6434,)

In [ ]:
#coverting the textual data to feature vectors
vectorizer = TfidfVectorizer()
# Convert x to a list of strings if it's not already
x = news_data['content'].tolist()  # or x = [str(item) for item in x] if x is a NumPy array
vectorizer.fit(x)
x = vectorizer.transform(x)

In [ ]:
print(x)

  (0, 177)	0.2766579900395751
  (0, 1498)	0.3943036065913846
  (0, 1781)	0.24877398104144344
  (0, 2158)	0.33987992132811795
  (0, 2274)	0.2675471171462601
  (0, 2956)	0.2351477798422862
  (0, 4169)	0.22182252043190626
  (0, 4568)	0.25193486007204036
  (0, 5114)	0.29650091018003516
  (0, 5281)	0.33987992132811795
  (0, 7941)	0.2580885725874065
  (0, 9269)	0.2845785714050832
  (1, 913)	0.28621032960610304
  (1, 1149)	0.15224483337184208
  (1, 1350)	0.3761194137731387
  (1, 1688)	0.1886609196257113
  (1, 2140)	0.27283241553682147
  (1, 3293)	0.7236601759089311
  (1, 4069)	0.1872863010460368
  (1, 9868)	0.290944438344452
  (2, 1769)	0.3207559383023785
  (2, 1875)	0.45642008971120857
  (2, 3221)	0.3840475510786025
  (2, 3572)	0.3410144523438057
  (2, 5690)	0.500306231102918
  :	:
  (6431, 2396)	0.20263947398421772
  (6431, 2537)	0.14668710689578726
  (6431, 2635)	0.32503057198874075
  (6431, 5960)	0.32503057198874075
  (6431, 6168)	0.18493838926712514
  (6431, 6877)	0.2318264986806305
  (6

Splitting the training and testing data

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

Training the Model : Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train,y_train)

LogisticRegression()

Evauation

accuracy score

In [ ]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('Accuracy score of the training data :' ,training_data_accuracy)

Accuracy score of the training data : 0.9782397513114436


In [ ]:
#accuracy score on the testing data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction,y_test)

Make a Predictive System

In [ ]:
x_news=x_test[0]
prediction = model.predict(x_news)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [ ]:
print(y_test[0])

0
